In [1]:
from glob import glob
import os
import sys
import re
import numpy as np
import pandas as pd
import pickle as pkl
import math
import json
import warnings
pd.set_option('display.max_columns',300)
pd.set_option('display.max_rows',300)
warnings.filterwarnings('ignore')

In [2]:
zhhh = pd.read_csv('wuhan_int_airport_outbound_Jan2020.csv')

In [3]:
def getToCountry(row):
    out = row.split(', ')
    return out[-1]

In [4]:
zhhh['To_country'] = zhhh['To'].apply(getToCountry)

### Top Outbound country from Wuhan Tianhe airport

In [6]:
zhhh.groupby(['From','To_country']).agg({'Id':'count'}).sort_values(by='Id', ascending = False)

Id
From                     To_country          
ZHHH Wuhan Tianhe, China China           3565
                         United Kingdom   953
                         Japan            518
                         Thailand         412
                         Malaysia         258
                         Macau            213
                         South Korea      170
                         Viet Nam         168
                         Hong Kong        167
                         Turkey           102
                         Singapore         71
                         United States     56
                         Taiwan            33

### Top Domestic Outbound City

In [8]:
zhhh[zhhh['To_country']=='China'].groupby(['From','To']).agg({'Id':'count'}).sort_values(by='Id', ascending = False)

Id
From                     To                                 
ZHHH Wuhan Tianhe, China ZBAA Beijing Capital, China     961
                         ZSPD Shanghai Pudong, China     913
                         ZGGG Guangzhou Baiyun, China    343
                         ZJHK Haikou Meilan, China       318
                         ZSSS Shanghai Hongqiao, China   229
                         ZYCC Longjia, Changchun, China  211
                         ZYHB Taiping, Harbin, China     176
                         ZUUU Chengdu Shuangliu, China   119
                         ZYTX Taoxian, Shenyang, China   103
                         ZSNB Ningbo Lishe, China         72
                         ZGSZ Shenzhen Bao'an, China      54
                         ZBLA Dongshan, Hailar, China     37
                         ZUCK Chongqing Jiangbei, China   15
                         ZPPP Kunming Wujiaba, China      12
                         ZSHC Hangzhou Xiaoshan, China     2

### Operator and Model of airplanes to Singapore

In [9]:
zhhh[zhhh['To']=='WSSS Singapore Changi, Singapore'].groupby('Op').agg({'Id':'count'}).sort_values(by='Id', ascending = False)

,Id
Op,
Scoot,71


In [10]:
zhhh[zhhh['To']=='WSSS Singapore Changi, Singapore'].groupby('Mdl').agg({'Id':'count'}).sort_values(by='Id', ascending = False)

,Id
Mdl,
Boeing 787 9,71


### Estimated number of passengers inbound from Wuhan (Direct flight) to Singapore

In [13]:
zhhh[zhhh['To']=='WSSS Singapore Changi, Singapore'].groupby('Mdl').agg({'Id':'count'}).sort_values(by='Id', ascending = False).values[0][0]*280

19880